### Autoren:
Mat. Nr.: 2888214, zuständig für Aufg. 2, 3, 5 <br>
Mat. Nr.: 3626401, zuständig für Aufg. 1, 4, 6

## Aufg. 1: Business Understanding

### Zielsetzung
Das Ziel dieses Projekts ist es, fünf potenzielle Städte für einen sechsmonatigen Aufenthalt als Remote Worker in der Kreativbranche zu identifizieren. Der Fokus liegt dabei auf tropisch warmen Ländern, da diese nicht nur ein angenehmes Klima bieten, sondern auch einzigartige wirtschaftliche und infrastrukturelle Eigenschaften aufweisen.

### Ausgangssituation und Fragestellungen
Wir arbeiten mit dem globalen „Cost of Living“-Datensatz von Kaggle, der uns detaillierte Informationen über Lebenshaltungskosten in Städten weltweit liefert. Anhand dieses Datensatzes wollen wir untersuchen, wie sich Lebenshaltungskosten und Infrastruktur in tropisch warmen Ländern im Vergleich zu Regionen wie Europa und Nordamerika darstellen. Unsere beiden Personen ernähren sich gerne Gesund, und legen daher auch viel Wert auf die Preise von Früchten & Obst und lokalen Essen. Sie betreiben ebenfalls viel Sport und haben kein Auto, weshalb Kosten für Freizeitaktivitäten und Öffentliche Verkersmittel großen Einfluss haben.

### Aufgestellte Hypothesen

1. **Tropisch warme Länder sind häufig touristisch geprägt.**  
   Es wird angenommen, dass tropische Länder aufgrund ihrer klimatischen und geografischen Merkmale oft als touristische Destinationen fungieren. Der damit verbundene touristische Einfluss könnte sich auf die lokale Preisstruktur auswirken, insbesondere in den Bereichen Unterkunft, Gastronomie und Freizeit, sowie Sommerkleidung, Sportartikel und alkoholische Getränke, da diese dadurch Teurer werden könnten.

2. **Tropische Länder weisen tendenziell niedrigere Lebenshaltungskosten auf als Städte in Europa und Nordamerika.**  
   Diese Hypothese basiert auf der Annahme, dass strukturelle wirtschaftliche Unterschiede – wie geringere durchschnittliche Einkommen, niedrigere Kaufkraft und eingeschränkter Zugang zu globalen Märkten – dazu führen, dass die Lebenshaltungskosten in tropisch warmen Ländern im Mittel unter denen entwickelter Volkswirtschaften liegen.

3. **Ein höherer Grad an Infrastruktur geht mit höheren Lebenshaltungskosten einher.**  
   Es wird vermutet, dass Städte mit besser ausgebauter Infrastruktur – insbesondere im Hinblick auf Internetzugang, Nahverkehr und Energieversorgung – ein höheres Preisniveau aufweisen. Diese Zusammenhänge könnten durch Investitionskosten, höhere Standards und ein allgemein urbaneres Umfeld erklärt werden, das typischerweise mit höheren Lebenshaltungskosten verbunden ist.

4. **Höhere Durchschnittsgehälter korrelieren mit höheren Wohnkosten.**  
   Diese Hypothese basiert auf der Annahme, dass das Mietpreisniveau an die lokale Kaufkraft gekoppelt ist. In Regionen mit höherem Einkommensniveau ist es wahrscheinlicher, dass höhere Mietpreise durchgesetzt werden können. Dadurch ergibt sich eine direkte Korrelation zwischen Einkommen und Wohnkosten.

5. **Die Kosten für öffentliche Verkehrsmittel stehen im Verhältnis zum lokalen Einkommensniveau.**  
   Es wird angenommen, dass die Preise für den öffentlichen Nahverkehr in Ländern mit geringerer Kaufkraft ebenfalls niedriger sind. Subventionierte Tarife oder ein geringeres Dienstleistungsniveau könnten dazu beitragen, dass die Transportkosten sich stärker am ökonomischen Umfeld der Bevölkerung orientieren.

6. **Freizeitkosten sind ein indirekter Indikator für den urbanen Lebensstandard.**  
   Die Hypothese geht davon aus, dass Städte mit höheren Freizeitkosten – etwa für Fitnessstudios, Kinos oder Sportanlagen – tendenziell ein höheres allgemeines Preisniveau und damit einen höheren Lebensstandard aufweisen. Solche Kosten können als Marker für Konsumverhalten und infrastrukturelle Ausstattung dienen.


### Weiteres Vorgehen
Auf Basis dieser Hypothesen werden wir in der Datenexploration und -analyse prüfen, inwiefern diese Zusammenhänge bestehen. Die gewonnenen Erkenntnisse sollen als Entscheidungsgrundlage dienen, um letztlich fünf Städte auszuwählen, die sowohl kosteneffizient als auch attraktiv hinsichtlich Klima, Infrastruktur und touristischem Potenzial sind.

## Aufg. 2: Data Preperation 
### Datenbereinigung und Zuordnung von Ländern zu Kontinenten

Zur Vorbereitung der Analyse wurden zunächst mehrere Schritte zur Datenbereinigung und -anreicherung durchgeführt:

1. **Einlesen und Filtern der Daten**  
   Der Datensatz `cost-of-living_v2.csv` enthält weltweite Informationen zu Lebenshaltungskosten. Zunächst wurden alle Einträge mit schlechter Datenqualität (`data_quality == 0`) entfernt, um Verzerrungen in der Analyse zu vermeiden. Anschließend wurde der DataFrame nach Ländern alphabetisch sortiert und die Indizes zurückgesetzt.

2. **Abruf aktueller Länderdaten mit Regionen (Kontinenten)**  
   Um die Daten regional auszuwerten, wurde eine API (`https://www.apicountries.com/countries`) verwendet, die Informationen über Länder und deren zugehörige Weltregionen bereitstellt. Die Antwort wurde in ein Dictionary (`country_to_continent`) überführt, in dem jedem Ländernamen der entsprechende Kontinent zugeordnet wurde. Dabei wurden auch alternative Schreibweisen berücksichtigt (z. B. "USA" für "United States of America").

3. **Fuzzy Matching zur Verbesserung der Länder-Zuordnung**  
   Da Ländernamen im Datensatz nicht immer exakt mit den offiziellen API-Bezeichnungen übereinstimmen (z. B. "South Korea" vs. "Korea, Republic of"), wurde ein mehrstufiges Matching-Verfahren eingesetzt:
   - Zunächst wird geprüft, ob der Ländername direkt im Mapping enthalten ist.
   - Falls nicht, wird das erste Wort des Ländernamens geprüft (z. B. "South").
   - Falls auch das keinen Treffer ergibt, wird ein Fuzzy-Matching-Algorithmus (über das `thefuzz`-Paket) verwendet, der auf Ähnlichkeiten zwischen Zeichenfolgen basiert. Nur Matches mit einer Ähnlichkeit über 80 % werden akzeptiert.

4. **Anreicherung des Datensatzes mit Kontinentinformationen**  
   Durch Anwendung der Funktion `find_best_match` auf die `country`-Spalte des ursprünglichen Datensatzes wurde jeder Stadt bzw. jedem Eintrag ein zugehöriger Kontinent zugeordnet. Diese Information bildet die Grundlage für weitere Auswertungen nach geografischen Regionen.

Diese Schritte stellen sicher, dass alle Länder konsistent klassifiziert sind und ermöglichen eine geografisch differenzierte Analyse der Lebenshaltungskosten weltweit.


In [ ]:
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from thefuzz import process

def find_best_match(country):
    # Versuche zuerst ein direktes Mapping
    if country in country_to_continent:
        return country_to_continent[country]

    # Versuche es mit dem ersten Wort
    first_word = country.split()[0]
    if first_word in country_to_continent:
        return country_to_continent[first_word]

    # Falls kein direkter Treffer: Fuzzy Matching versuchen
    best_match, score = process.extractOne(country, country_to_continent.keys())
    if score > 80:  # 80 bedeutet "ziemlich sicher"
        return country_to_continent[best_match]

    # Falls nichts gefunden, None zurückgeben
    return None

# Entferne die Spalten mit schlechter Qualität
data = pd.read_csv("./data/cost-of-living_v2.csv")
data.drop(data[data['data_quality'] == 0].index, inplace=True)
data.sort_values(by=['country'], inplace=True)
data.reset_index(drop=True, inplace=True)


# Mappen der Länder zu Kontinenten
url = "https://www.apicountries.com/countries"
response = requests.get(url)
countries = response.json()

country_to_continent = {}
for item in countries:
    country_to_continent[item["name"]] = item["region"]

    if "altSpellings" in item:
        for alt_name in item["altSpellings"]:
            country_to_continent[alt_name] = item["region"]

data["continent"] = data["country"].apply(find_best_match)


## Aufg. 3: Data Exploration und Analyse
Untersuchen Sie den Datensatz in Bezug auf das Ziel nach den Regeln wie in der Vorlesung gelehrt. Nutzen Sie Markup, um wichtige Erkenntnisse zu dokumentieren.

Werten Sie sowohl Kontinente, Länder, Städte sinnvoll nach Ihren Zielen generell aus. Berücksichtigen Sie ihr Gruppenziel dabei spezifisch.

In [ ]:
column_names = {
    'x16': 'Apples (1kg) (USD)',
    'x14' : 'Chicken Fillets (1kg) (USD)',
    'x28': 'ÖPNV Einzelfahrt',
    'x29': 'Monatsticket',
    'x30': 'Taxi Start',
    'x31': 'Taxi 1km',
    'x39': 'Fitnessstudio (monatlich)',
    'x40': 'Tennisplatz (1h, Wochenende)',
    'x41': 'Kino (1 Ticket)',
    'x51': 'Apartment (3 bedrooms) Outside of Centre (USD)',
    'x54': 'Average Monthly Net Salary (After Tax) (USD)'
}

fig, axes = plt.subplots(1, 4, figsize=(30, 10))
numeric_columns = ['x14','x16', 'x51', 'x54']

for ax, column in zip(axes, numeric_columns):
    sns.boxplot(x='continent', y=column, data=data, ax=ax)
    ax.set_title(f"{column_names[column]} nach Kontinent")


plt.tight_layout()
plt.show()

### Analyse der Lebenshaltungskosten nach Kontinenten

Im Rahmen dieser Untersuchung wurden zentrale Ausgabenkategorien nach Kontinent differenziert analysiert, um die Eignung tropischer Regionen für Remote Work fundiert zu bewerten. Im Fokus standen Variablen, die für einen gesundheitsbewussten Lebensstil von besonderer Bedeutung sind – darunter Lebensmittelpreise, Wohnkosten sowie das durchschnittliche Nettoeinkommen.

#### Fleischpreise – Beispiel: Hähnchenbrustfilet (`x14`)
Die Analyse zeigt, dass insbesondere Europa ein signifikant höheres Preisniveau für Fleischprodukte wie Hähnchenbrustfilet aufweist. Demgegenüber bieten tropische Regionen wie Asien, Afrika und Teile Südamerikas deutlich niedrigere Preise. Dies stellt einen klaren Vorteil dar, wenn eine proteinreiche, ausgewogene Ernährung angestrebt wird – ein Aspekt, der für beruflich aktive Personen mit Fokus auf Gesundheit besonders relevant ist.

#### Obstpreise – Beispiel: Äpfel (`x16`)
Ein vergleichbares Bild ergibt sich bei Obst: Während Europa die höchsten Apfelpreise aufweist, liegen die Medianwerte in Asien und Afrika erheblich darunter. Diese Unterschiede deuten darauf hin, dass eine frische, vitaminreiche Ernährung in tropischen Ländern mit geringeren finanziellen Aufwendungen realisierbar ist – ein weiterer Pluspunkt bei der Wahl eines temporären Wohnortes im globalen Süden.

#### Wohnkosten – Beispiel: 3-Zimmer-Wohnung außerhalb des Zentrums (`x51`)
Die Wohnkosten in tropischen Regionen fallen im Vergleich zu Europa und Nordamerika signifikant niedriger aus. Dies gilt sowohl für Mietpreise im Stadtzentrum als auch in peripheren Lagen. Die geringeren Mietkosten ermöglichen es, größeren Wohnraum zu günstigeren Konditionen zu mieten – ein erheblicher Vorteil bei längeren Auslandsaufenthalten und insbesondere dann, wenn Remote Work von zu Hause aus erfolgt.

#### Einkommen – Durchschnittliches Nettoeinkommen (`x54`)
Das durchschnittliche monatliche Nettoeinkommen ist in Europa und Nordamerika deutlich höher als in tropischen Ländern. Dieser Unterschied ist jedoch im Kontext dieser Untersuchung von untergeordneter Bedeutung, da Remote Worker ihr Einkommen nicht lokal generieren. Vielmehr ergibt sich hieraus ein strategischer Vorteil: Ein international konkurrenzfähiges Einkommen lässt sich in tropischen Ländern aufgrund der niedrigeren Lebenshaltungskosten effektiver nutzen.


In [ ]:
transport_columns = ['x28',
                     'x29',
                     'x30',
                     'x31'
]

fig, axes = plt.subplots(1, 5, figsize=(35, 10))
transport_columns = ['x28', 'x29', 'x30', 'x31','x54']

for ax, column in zip(axes, transport_columns):
    sns.boxplot(x='continent', y=column, data=data, ax=ax)
    ax.set_title(f"{column_names[column]} nach Kontinent")
    ax.set_xlabel("Kontinent")
    ax.set_ylabel("Kosten (USD)")

plt.tight_layout()
plt.show()


### Analyse der Mobilitätskosten nach Kontinent

Die grafische Auswertung stellt die Kosten verschiedener Mobilitätsformen – darunter Einzelfahrten im öffentlichen Nahverkehr, Monatstickets sowie Taxikosten (Starttarif und Preis pro Kilometer) – im interkontinentalen Vergleich dar. Auffällig ist, dass insbesondere **asiatische und afrikanische Städte** ein vergleichsweise niedriges Preisniveau in allen betrachteten Mobilitätskategorien aufweisen.

Bei **Einzelfahrten im ÖPNV** liegen die Medianwerte in Asien und Afrika deutlich unter denen in Europa, den Amerikas und Ozeanien. Noch deutlicher zeigt sich dieser Trend bei **Monatstickets**, bei denen die langfristigen Mobilitätskosten in Europa und Ozeanien rund doppelt so hoch ausfallen wie in Afrika oder Asien. Auch die **Taxikosten** – sowohl beim Starttarif als auch beim Preis pro Kilometer – folgen demselben Muster: Europa ist hier durchweg teurer, während asiatische und afrikanische Städte günstige Alternativen bieten.

### Einordnung im Kontext des Projekts

Für Remote Worker, die im Zielland kein eigenes Fahrzeug nutzen möchten, ist eine erschwingliche Mobilität entscheidend. Die Ergebnisse deuten darauf hin, dass insbesondere Städte in **Asien und Afrika** deutliche Kostenvorteile bei Alltagsmobilität bieten. Diese Aspekte sind besonders für eine sechsmonatige Aufenthaltsplanung von Bedeutung, da sie fixe Ausgaben betreffen, die regelmäßig anfallen. Die niedrigeren Kosten in diesen Regionen können somit zu einer effizienteren Budgetplanung beitragen und die Lebensqualität vor Ort erhöhen.


In [ ]:
freizeit_columns = ['x39',
                    'x40',
                    'x41']

fig, axes = plt.subplots(1, 3, figsize=(24, 10))
freizeit_columns = ['x39', 'x40', 'x41']

for ax, column in zip(axes, freizeit_columns):
    sns.boxplot(x='continent', y=column, data=data, ax=ax)
    ax.set_title(f"{column_names[column]} nach Kontinent")
    ax.set_xlabel("Kontinent")
    ax.set_ylabel("Kosten (USD)")

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Gruppierung: Durchschnittswerte pro Land (und Aufnahme des Kontinents)
# Nur numerische Spalten berücksichtigen
numeric_columns = data.select_dtypes(include=['number']).columns
country_data = data.groupby(['country', 'continent'])[numeric_columns].mean().reset_index()

# Liste der zu plottenden Variablen
vars_to_plot = ['x16', 'x28', 'x39', 'x51']
titles = {
    'x16': 'Apfelpreis (1kg, USD)',
    'x28': 'Einzelticket ÖPNV (USD)',
    'x39': 'Fitnessstudio (Monatsgebühr, USD)',
    'x51': 'Apartment (3 bedrooms outside Centre, USD)'
}

# Erstelle ein 2x2-Grid für die Barplots mit erhöhter Höhe, um alle Ländernamen lesbar darzustellen.
fig, axes = plt.subplots(2, 2, figsize=(20, 30))

for ax, var in zip(axes.flatten(), vars_to_plot):
    # Sortieren nach dem jeweilig betrachteten Merkmal
    plot_data = country_data.sort_values(by=var, ascending=False)
    
    # Barplot: x-Achse mit den Werten, y-Achse mit den Ländern, Balken nach Kontinent eingefärbt.
    sns.barplot(data=plot_data, x=var, y='country', hue='continent', dodge=False, ax=ax)
    ax.set_title(titles[var])
    ax.set_xlabel(titles[var])
    ax.set_ylabel("Land")
    ax.legend(loc='best', title='Kontinent')
    
    # Länder-Namen auf der y-Achse horizontal ausrichten und ggf. Schriftgröße anpassen.
    for label in ax.get_yticklabels():
        label.set_rotation(0)
    ax.tick_params(axis='y', labelsize=8)

plt.tight_layout()
plt.show()

### Visualisierung der Durchschnittskosten pro Land

Diese Darstellung umfasst vier horizontale Barplots in einem 1×4-Layout. Für jede Kostenvariable wird der durchschnittliche Wert pro Land (x-Achse) angezeigt. Dabei sind die Balken farblich nach Kontinent kodiert, sodass regionale Unterschiede auf einen Blick erkennbar werden.

- **x16:** Apfelpreis (1kg, USD)  
- **x28:** Kosten für ein Einzelticket im ÖPNV (USD)  
- **x39:** Monatsgebühr für ein Fitnessstudio (USD)  
- **x51:** Mietkosten für ein Apartment (3 Bedrooms, außerhalb des Zentrums, USD)

Man sieht, dass in den obersten Rängen bei den Plots meist Europa, Americas oder Ozeanien liegen.

### Analyse der Freizeit- und Sportkosten nach Kontinent

Die grafische Darstellung analysiert drei zentrale Kostenfaktoren für die Freizeitgestaltung: monatliche Mitgliedschaften im Fitnessstudio, die stündliche Miete eines Tennisplatzes am Wochenende sowie den Preis für ein Kinoticket. Diese Ausgaben spielen eine zentrale Rolle für Personen, die auch im Ausland einen aktiven, gesundheitsorientierten Lebensstil pflegen möchten.

Im Bereich der **Fitnessstudio-Kosten** zeigt sich, dass der Median in Asien zwar moderat ausfällt, die Spannweite der Preise jedoch sehr hoch ist. Das bedeutet, dass neben günstigen auch sehr teure Fitnessangebote existieren. Afrika und die Amerikas bieten vergleichbare Medianwerte, während Europa und insbesondere Ozeanien insgesamt höhere Kosten aufweisen.

Die **Mietkosten für Tennisplätze** sind am stärksten in Europa ausgeprägt, gefolgt von Ozeanien. Afrika und Asien liegen deutlich darunter, was nahelegt, dass sportliche Aktivitäten im Freien dort erschwinglicher sind. In Afrika ist zudem die Varianz besonders gering, was auf ein stabiles Preisniveau hindeutet.

Bei den **Kinopreisen** ergibt sich ein klareres Bild: In Afrika und Asien sind die Ticketpreise im Durchschnitt am niedrigsten. Europa, die Amerikas und Ozeanien bewegen sich hingegen auf einem höheren Niveau, mit Europa als Spitzenreiter in Bezug auf die Preisverteilung.

### Interpretation im Hinblick auf das Projektziel

Für Remote Worker, die auch im Ausland nicht auf regelmäßige sportliche und kulturelle Aktivitäten verzichten möchten, sind diese Kosten ein relevanter Faktor bei der Auswahl eines geeigneten Standorts. Die Analyse zeigt, dass vor allem **asiatische und afrikanische Städte** ein günstiges Preisniveau in diesen Bereichen aufweisen – sowohl im Fitness- als auch im Kulturbereich. Damit bieten diese Regionen nicht nur klimatische und wirtschaftliche Vorteile, sondern auch finanziellen Spielraum für eine ausgeglichene Work-Life-Balance.


In [ ]:
infrastructure = data.copy()
infrastructure['infrastruktur_index'] = infrastructure[['x36', 'x38']].mean(axis=1).copy()

plt.figure(figsize=(10, 6))
sns.regplot(data=infrastructure, x='infrastruktur_index', y='x50', scatter_kws={'alpha': 0.6}, line_kws={'color': 'red'})
plt.title('Zusammenhang zwischen Infrastrukturkosten und Miete (Zentrum)')
plt.xlabel('Infrastrukturkosten (Strom + Internet, Ø USD)')
plt.ylabel('Miete (3-Zimmer-Wohnung im Zentrum, USD)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### Hypothese 3:
> **Ein höherer Grad an Infrastruktur geht mit höheren Lebenshaltungskosten einher.**  
> Es wird vermutet, dass Städte mit besser ausgebauter Infrastruktur – insbesondere im Hinblick auf Internetzugang, Nahverkehr und Energieversorgung – ein höheres Preisniveau aufweisen. Diese Zusammenhänge könnten durch Investitionskosten, höhere Standards und ein allgemein urbaneres Umfeld erklärt werden, das typischerweise mit höheren Lebenshaltungskosten verbunden ist.

### Analyse des Scatterplots  
Zur Überprüfung dieser Hypothese wurde ein Scatterplot erstellt, der die durchschnittlichen Infrastrukturkosten – gebildet aus den monatlichen Ausgaben für Strom und Internet – den Mietpreisen für eine 3-Zimmer-Wohnung im Stadtzentrum gegenüberstellt. Die grafische Darstellung zeigt einen insgesamt positiven Trend: Städte mit höheren Ausgaben für Strom und Internet tendieren auch zu höheren Mietpreisen im Zentrum. Die eingefügte Regressionslinie unterstreicht diesen linearen Zusammenhang, auch wenn eine gewisse Streuung in den Daten erkennbar bleibt. Diese Streuung deutet darauf hin, dass regionale Besonderheiten oder andere Faktoren – etwa politische Rahmenbedingungen, Subventionen oder wirtschaftliche Struktur – ebenfalls eine Rolle spielen können.

### Einordnung in die Hypothese  
Die Ergebnisse der Visualisierung liefern eine gute empirische Unterstützung für Hypothese 3. Der beobachtete Zusammenhang spricht dafür, dass infrastrukturell stärker ausgebaute Städte ein höheres allgemeines Preisniveau aufweisen – zumindest in Bezug auf Strom, Internet und zentrale Mietkosten. Auch inhaltlich passt diese Beobachtung zu den Annahmen: Infrastrukturausbau erfordert Investitionen, die sich möglicherweise in Form höherer Kosten für die Bevölkerung widerspiegeln.

### Kritische Bewertung  
Obwohl der Zusammenhang grundsätzlich erkennbar ist, lässt sich keine perfekte lineare Beziehung feststellen. Die Verteilung der Datenpunkte zeigt, dass es durchaus Ausreißer gibt, bei denen hohe Infrastrukturkosten nicht zwangsläufig mit entsprechend hohen Mieten einhergehen – und umgekehrt. Solche Abweichungen könnten auf lokale Besonderheiten zurückzuführen sein, etwa stark subventionierte Lebenshaltungskosten, regionale Preisregulierungen oder Unterschiede in der wirtschaftlichen Leistungsfähigkeit. Zudem muss betont werden, dass es sich bei dem Scatterplot um eine Korrelationsanalyse handelt: Ein statistischer Zusammenhang lässt sich zwar beobachten, daraus kann jedoch keine kausale Beziehung abgeleitet werden.

Ein weiterer Aspekt betrifft mögliche regionale Unterschiede. Es ist denkbar, dass der Zusammenhang in bestimmten Teilen der Welt – etwa in Südostasien oder Westeuropa – stärker oder schwächer ausgeprägt ist. Eine weiterführende Analyse könnte daher darin bestehen, die Korrelation für einzelne Kontinente oder Einkommensgruppen gesondert zu betrachten.  
### Fazit  
Insgesamt bestätigt der Scatterplot die grundlegende Annahme der dritten Hypothese: Ein höheres infrastrukturelles Niveau scheint mit höheren Mietpreisen zu korrelieren. Zwar zeigt sich keine strenge lineare Beziehung, doch die Tendenz ist deutlich und unterstützt die Idee, dass moderne, gut ausgebaute Städte in vielen Fällen auch mit höheren Lebenshaltungskosten verbunden sind.


In [ ]:
travel = data.copy()
travel['travel_index'] = travel[['x28','x29']].mean(axis=1).copy()

plt.figure(figsize=(20, 8))
sns.regplot(data=travel, x='travel_index', y='x54', scatter_kws={'alpha': 0.6}, line_kws={'color': 'red'})
plt.title('Zusammenhang zwischen Reisekosten und Nettoeinkommen')
plt.xlabel('Kosten ÖPNV, Ø USD)')
plt.ylabel('Nettoeinkommen, USD)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### Hypothese 5:
> **Die Kosten für öffentliche Verkehrsmittel stehen im Verhältnis zum lokalen Einkommensniveau.**  
> Es wird angenommen, dass die Preise für den öffentlichen Nahverkehr in Ländern mit geringerer Kaufkraft ebenfalls niedriger sind. Subventionierte Tarife oder ein geringeres Dienstleistungsniveau könnten dazu beitragen, dass sich die Transportkosten stärker am ökonomischen Umfeld der Bevölkerung orientieren.

### Analyse des Scatterplots  
Der Scatterplot visualisiert den Zusammenhang zwischen den durchschnittlichen Kosten für den öffentlichen Nahverkehr (Einzelticket und Monatsticket) und dem durchschnittlichen Nettoeinkommen pro Stadt. Der Trend wird durch eine Regressionslinie mit Konfidenzintervall dargestellt.

Die Darstellung zeigt einen deutlich positiven Zusammenhang: Städte mit höheren ÖPNV-Kosten weisen tendenziell auch ein höheres monatliches Nettoeinkommen auf. Diese Beobachtung bestätigt die Erwartung, dass sich Preise für den öffentlichen Nahverkehr häufig am ökonomischen Umfeld der Bevölkerung orientieren. In Ländern mit geringerer Kaufkraft sind die ÖPNV-Tarife entsprechend niedriger, sei es durch staatliche Subventionen, geringere Lohnkosten oder einfach durch ein reduziertes Dienstleistungsangebot.

### Einordnung in die Hypothese  
Die Ergebnisse des Scatterplots stützen Hypothese 5 klar. Der beobachtete Trend zeigt, dass die Preise für Bus, Bahn oder U-Bahn mit dem Einkommensniveau steigen. Dies legt nahe, dass der öffentliche Verkehr häufig relativ erschwinglich zur Kaufkraft der lokalen Bevölkerung bleibt.

### Kritische Bewertung  
Obwohl ein positiver Trend deutlich erkennbar ist, gibt es – wie auch im Plot sichtbar – Ausreißer, bei denen etwa niedrige ÖPNV-Kosten mit relativ hohen Einkommen oder umgekehrt einhergehen. Solche Abweichungen könnten auf unterschiedliche Subventionsstrategien, Tarifmodelle oder Infrastrukturqualität zurückzuführen sein.

Zudem muss betont werden, dass es sich hierbei um eine Korrelation und keine Kausalität handelt. Der Trend liefert einen Hinweis auf einen Zusammenhang, beweist jedoch nicht, dass höhere Einkommen die Ursache für höhere ÖPNV-Preise sind (oder umgekehrt).

### Fazit  
Der Plot liefert eine fundierte empirische Unterstützung für Hypothese 5. Die Ergebnisse legen nahe, dass sich die Kosten des öffentlichen Nahverkehrs im internationalen Vergleich am lokalen Einkommen orientieren und somit Teil eines sozialverträglichen Preisniveaus sind. Dies macht den ÖPNV zu einem aussagekräftigen Indikator für die wirtschaftliche Struktur und das Preisgefüge einer Stadt.

In [ ]:
# Hypothese 6
# Berechne einen Freizeit-Index aus den relevanten Freizeitkosten (x39, x40, x41)
data_copy = data.copy()
data_copy['freizeit_index'] = data_copy[['x39', 'x40', 'x41']].mean(axis=1)

# Erstelle einen Scatterplot mit Regressionslinie: Freizeitkosten vs. Durchschnittsgehalt
plt.figure(figsize=(12, 6))
sns.regplot(data=data_copy, x='freizeit_index', y='x54', 
            scatter_kws={'alpha': 0.6}, line_kws={'color': 'red'})
plt.title('Zusammenhang zwischen Freizeitkosten und Nettoeinkommen')
plt.xlabel('Freizeit Index (Durchschnittliche Freizeitkosten, USD)')
plt.ylabel('Average Monthly Net Salary (x54, USD)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### Hypothese 6: Freizeitkosten als Indikator für urbanen Lebensstandard

**Hypothese:**  
Städte mit höheren Freizeitkosten (z. B. für Fitnessstudios, Tennisplätze oder Kinobesuche) weisen tendenziell auch ein höheres allgemeines Preisniveau und Einkommen auf. Diese Freizeitkosten können somit als Indikator für ein gehobenes urbanes Umfeld verstanden werden, das sowohl Konsumspielraum als auch infrastrukturelle Angebote widerspiegelt.

**Vorgehen:**  
Ein **Freizeit-Index** wurde als Durchschnitt der folgenden Kostenindikatoren berechnet:
- x39: Monatsbeitrag Fitnessstudio  
- x40: 1 Stunde Tennisplatzmiete  
- x41: Kinoticket  

Dieser Index wurde gegen das **durchschnittliche monatliche Nettoeinkommen** (x54) aufgetragen, um zu untersuchen, ob ein statistischer Zusammenhang zwischen den Freizeitkosten und dem verfügbaren Einkommen besteht.

**Visualisierung:**  
Der Scatterplot zeigt den Zusammenhang zwischen dem Freizeit-Index und dem Nettoeinkommen. Eine Regressionslinie mit Konfidenzintervall ist ebenfalls dargestellt:
**Interpretation:**  
Der Plot verdeutlicht einen **klar positiven linearen Zusammenhang**: Höhere Freizeitkosten treten häufiger in Städten mit höherem Einkommen auf. Dies spricht dafür, dass Freizeitpreise ein brauchbarer Indikator für die ökonomische Gesamtlage und den Lebensstandard in Städten sein können. Die Varianz ist insbesondere bei mittleren Freizeitkosten hoch, was auf regionale Unterschiede und Lebenshaltungsstrukturen hindeuten könnte, doch der generelle Trend unterstützt die Hypothese.

**Fazit:**  
Die Hypothese, dass Freizeitkosten als Indikator für urbanen Lebensstandard genutzt werden können, wird durch diese Analyse unterstützt.

In [ ]:
data.fillna(0, inplace=True)
data.drop(columns=['data_quality'], inplace=True)
corr = data.drop(columns=['city', 'country', 'continent']).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(30, 15))
sns.heatmap(corr, mask=mask, cmap="coolwarm", center=0, annot=True, fmt=".2f", linewidths=0.5, cbar_kws={"shrink": 0.5})
plt.title("Korrelationsmatrix der Daten")
plt.show()

### Analyse der Korrelationen im Gesamtdatensatz

Zur fundierten Bewertung der aufgestellten Hypothesen wurde eine Korrelationsmatrix erstellt, die Zusammenhänge zwischen den verschiedenen Lebenshaltungskosten, Infrastruktur- und Einkommensvariablen aufzeigt. Diese erlaubt es, erste Indikatoren für systematische Zusammenhänge zu identifizieren und zu prüfen, inwiefern sich die Hypothesen im Datenmaterial widerspiegeln.

---

#### Zusammenhang von Restaurant- und Lebensmittelpreisen

Innerhalb der Variablenbereiche `x1` bis `x6` (Preise für Restaurantbesuche) zeigt sich eine sehr hohe Korrelation (z. B. `x1` zu `x2`: 0.88), ebenso bei den Supermarktpreisen zwischen `x9` und `x21`, etwa `x13` zu `x14` mit 0.72. Das deutet darauf hin, dass Preisniveaus in diesen Bereichen eng miteinander verknüpft sind: In Städten mit teuren Restaurants sind in der Regel auch Supermarktprodukte kostenintensiver. Diese Beobachtung unterstützt indirekt Hypothese 1, indem sie nahelegt, dass touristisch geprägte Orte – die häufig über ein hohes Gastronomieangebot verfügen – auch in anderen Lebensbereichen ein erhöhtes Preisniveau aufweisen.

---

#### Einkommen im Verhältnis zu Wohnkosten

Ein zentrales Ergebnis der Korrelationsanalyse ist die starke Verbindung zwischen dem durchschnittlichen Nettoeinkommen (`x54`) und den Wohnkosten. Besonders auffällig ist die hohe Korrelation zu den Mietpreisen für große Wohnungen im Stadtzentrum (`x50` = 0.86), außerhalb des Zentrums (`x51` = 0.73) sowie zu den Wohnungskaufpreisen je Quadratmeter (`x52` = 0.83 im Zentrum, `x53` = 0.72 außerhalb). Diese Werte stützen Hypothese 4 und legen nahe, dass ein höheres Einkommensniveau auch mit höheren Wohnkosten einhergeht. Dieses Muster ist plausibel, da Städte mit hoher Kaufkraft auch höhere Mietpreise durchsetzen können – entweder durch höhere Nachfrage, oder durch ein besseres, teureres Wohnangebot. Der Zusammenhang reflektiert zudem den Entwicklungsgrad einer Stadt und ihre wirtschaftliche Struktur.

---

#### Einkommen und sonstige Lebenshaltungskosten

Auch bei anderen Lebenshaltungskosten zeigt sich eine moderate Korrelation mit dem Einkommen. So liegen die Korrelationswerte zwischen `x54` und Lebensmitteln oder Restaurants im Bereich von 0.3 bis 0.6. Dieses Ergebnis spricht für Hypothese 2 und 5, wonach sich wirtschaftlich stärkere Städte auch durch ein insgesamt höheres Preisniveau auszeichnen. In ärmeren Regionen hingegen sind sowohl Gehälter als auch Kosten für alltägliche Güter niedriger – eine wichtige Information für Remote Worker, deren Gehalt unabhängig vom lokalen Arbeitsmarkt ist. Für diese Zielgruppe ergibt sich daraus ein potenzieller Vorteil, da niedrigere Lebenshaltungskosten bei konstantem Einkommen den Lebensstandard effektiv erhöhen.

---

#### Rolle der Internetpreise

Ein bemerkenswerter Befund ist die vergleichsweise geringe Korrelation der Internetkosten (`x38`) mit den übrigen Variablen. Weder zu Mieten, Gehältern noch zu Infrastrukturvariablen besteht ein klarer Zusammenhang. Diese Unabhängigkeit macht Internetkosten zu einer eigenständigen Größe im Rahmen der Stadtbewertung – insbesondere für digitale Nomaden, die auf eine verlässliche und erschwingliche Internetverbindung angewiesen sind. In Hinblick auf Hypothese 3 bedeutet dies, dass nicht alle Infrastrukturelemente automatisch das allgemeine Preisniveau beeinflussen.

---

### Gesamtbewertung der Korrelationen im Kontext der Hypothesen

Die Korrelationsmatrix liefert eine breite empirische Basis zur Unterstützung mehrerer Hypothesen:

- **Hypothese 1 (Touristische Prägung)**: Wird gestützt durch stark korrelierende Preise in Gastronomie und Freizeit.
- **Hypothese 2 (Geringere Lebenshaltungskosten in Tropen)**: Moderate Korrelationen zeigen globale Muster zwischen Einkommen und Preisniveau.
- **Hypothese 3 (Infrastruktur & Kosten)**: Nur teilweise gestützt, da z. B. Internetkosten unabhängig von Mieten bleiben.
- **Hypothese 4 (Gehalt vs. Wohnkosten)**: Wird durch starke positive Korrelationen eindeutig unterstützt.
- **Hypothese 5 (ÖPNV vs. Einkommen)**: Korrelationen mit ÖPNV-Kosten wurden ergänzend visuell untersucht.
- **Hypothese 6 (Freizeitkosten als Indikator)**: Fitness- und Kinokosten korrelieren mittelstark mit Einkommen und anderen Preisen und könnten somit als Proxy für urbanen Lebensstandard dienen.

Die Ergebnisse bestätigen, dass viele Preisniveaus nicht isoliert auftreten, sondern in systematische Zusammenhänge eingebettet sind. Daraus ergeben sich wichtige Anhaltspunkte für eine faktenbasierte Entscheidungsfindung im Kontext von Remote Work und Standortwahl.


In [ ]:
from bs4 import BeautifulSoup

url = "https://worldpopulationreview.com/country-rankings/tropical-countries"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", class_="wpr-table min-w-full border-collapse")
rows = table.find_all("tr")
tropical_countries = []
for row in rows:
    cells = row.find_all("td")
    if len(cells) > 0:
        if "Fully Tropical" == cells[2].text.strip():
            tropical_countries.append(cells[1].text.strip())
print(len(data))
data_tropical = data[data["country"].isin(tropical_countries)].copy()
print(len(tropical_countries) - len(data_tropical), len(data_tropical))
print(set(tropical_countries) - set(data["country"].unique()))

Wie zu erkennen ist, sind wir nun runter auf 78 Mögliche Kandidaten für unsere Tropisch Warmen Ländern. Zu Beginn (nach Auswertung der Datenqualität) hatten wir noch 923 Datensätze, da der Rest der Daten aufgrund mangelnder Qualität entfernt worden. Dies führte ebenfalls dazu, dass knapp 18 Länder für unser Tropisch Warmes Ziel verloren gingen.

In [ ]:
# Kategorien für touristische Einflüsse
features_h1 = ['x1', 'x2', 'x4', 'x5', 'x24', 'x39', 'x40', 'x41', 'x44', 'x45', 'x46']

# Mittelwerte berechnen
avg_tropisch = data_tropical[features_h1].mean()
avg_global = data[features_h1].mean()

# Differenz berechnen
differenz = avg_tropisch - avg_global

# Plot
plt.figure(figsize=(12, 6))
differenz.plot(kind='bar', color='coral', edgecolor='black')
plt.axhline(0, color='gray', linewidth=1)
plt.title("Preisdifferenz Tropisch vs. Global (touristische Kategorien)")
plt.ylabel("Differenz (tropisch - global) in USD")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### Hypothese 1  
> **Tropisch warme Länder sind häufig touristisch geprägt.**  
>Es wird angenommen, dass tropische Länder aufgrund ihrer klimatischen und geografischen Merkmale oft als touristische Destinationen fungieren. Der damit verbundene touristische Einfluss könnte sich auf die lokale Preisstruktur auswirken, insbesondere in den Bereichen Unterkunft, Gastronomie und Freizeit, sowie Sommerkleidung, Sportartikel und alkoholische Getränke, da diese dadurch Teurer werden könnten.

---

### Analyse des Differenzdiagramms  
Zur Untersuchung dieser Hypothese wurden verschiedene Preisindikatoren ausgewählt, die typischerweise in touristischen Kontexten eine Rolle spielen – darunter Restaurantbesuche (`x1`, `x2`), alkoholische Getränke (`x4`, `x5`, `x24`), Freizeitangebote (`x39`, `x40`, `x41`) sowie Kleidung und Schuhe (`x44`, `x45`, `x46`). Der Balkenplot zeigt die mittlere Preisdifferenz zwischen tropischen Ländern und dem globalen Durchschnitt für diese Kategorien.

Auffällig ist, dass die Mehrheit der Balken **negative Werte aufweist**. Dies bedeutet, dass in tropischen Ländern die Preise für die entsprechenden Güter und Dienstleistungen **niedriger** sind als im globalen Mittel. Besonders stark ist dieser Effekt in den Bereichen **Gastronomie (`x2`)**, **Sommerkleidung (`x44`, `x45`)** und **Sportschuhe (`x46`)** zu beobachten – alles typische Konsumgüter in touristisch geprägten Regionen.

Lediglich in vereinzelten Kategorien – etwa bei **Wein (`x24`)** – sind leicht positive Abweichungen erkennbar. Diese könnten auf importabhängige Produkte oder regionale Marktbesonderheiten zurückzuführen sein.

---

### Einordnung in die Hypothese  
Die dargestellten Preisdifferenzen stützen die Hypothese insofern, als dass tropische Länder – trotz ihrer touristischen Relevanz – **nicht grundsätzlich teurer** sind in den analysierten Konsumfeldern. Vielmehr zeigen sich dort **niedrigere Preise**, was einerseits auf das lokale Lohn- und Preisniveau zurückzuführen sein kann, andererseits aber auch auf eine gezielte Preisanpassung an touristische Nachfrage in bestimmten Marktsegmenten.

Es ist möglich, dass tropische Länder durch geringere Produktionskosten, niedrigere Besteuerung oder staatliche Subventionen in der Lage sind, zentrale Konsumgüter – wie Essen, Kleidung oder Freizeitangebote – kostengünstig anzubieten, selbst wenn sie touristisch stark frequentiert sind.

---

### Kritische Bewertung  
Der Plot zeigt, dass ein touristischer Einfluss **nicht zwangsläufig** mit höheren Preisen einhergeht – im Gegenteil: Viele tropische Regionen scheinen günstiger zu sein, obwohl sie touristisch attraktiv sind. Ein Erklärungsansatz könnte darin liegen, dass sich der Tourismuseffekt auf andere, hier nicht erfasste Segmente auswirkt, etwa auf **Luxusgüter, Unterkunftspreise in Spitzenlagen oder saisonal schwankende Dienstleistungen**.

Zudem könnte die durchschnittliche Preisbetrachtung durch Regionen mit sehr niedrigen Lebenshaltungskosten (z. B. ländliche Gebiete in tropischen Ländern) verzerrt sein. Eine weitere Untersuchung auf Basis differenzierterer regionaler Kategorien (z. B. touristische Küstenstädte vs. Inland) könnte hier weitere Erkenntnisse liefern.

---

### Fazit  
Insgesamt zeigt sich, dass tropische Länder in touristisch geprägten Konsumbereichen **tendenziell günstiger** sind als der globale Durchschnitt. Die Hypothese, dass touristischer Einfluss zu Preissteigerungen führt, lässt sich mit den hier untersuchten Daten **nicht klar bestätigen**. Stattdessen deuten die Ergebnisse darauf hin, dass tropische Regionen attraktive Konsumpreise bieten – was sie sowohl für Touristen als auch für längere Aufenthalte (z. B. Remote Work) wirtschaftlich interessant macht.


In [ ]:
relevant_cols = ['x48','x49','x50','x51','x52','x53','x54']
tropical_avg = data_tropical[relevant_cols].mean()
global_avg = data[relevant_cols].mean()
diff = tropical_avg - global_avg
diff.plot(kind='bar', color='coral', figsize=(12, 6), edgecolor='black')
plt.axhline(0, color='black', linewidth=0.8)
plt.title('Differenz: Tropical Average - Global Average (Mietkosten, + Werte Tropische Länder haben mehr / - Werte Tropische Länder haben weniger)')
plt.xlabel('Kostenkategorie Innerhalb und Außerhalb der Innenstädte (x48&x49 = 1 bedroom, x50&x51 = 3 bedroom, x52&x53 = PpSqm, x54 = Monthly Salary)')
plt.ylabel('Differenz')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Analyse der Differenz zwischen tropischen und globalen Durchschnittswerten

Ein wesentlicher Bestandteil der Standortbewertung betrifft die Wohnkosten und das lokale Einkommensniveau. Die zugehörige Grafik veranschaulicht die Differenz zwischen tropischen Ländern und dem globalen Durchschnitt hinsichtlich Mieten, Immobilienpreisen und Nettoeinkommen.

Die Ergebnisse zeigen ein konsistentes Bild: Die **Mietkosten** in tropischen Ländern sind durchgehend **deutlich niedriger** als im weltweiten Vergleich. Dies gilt sowohl für kleinere Wohnungen mit einem Schlafzimmer als auch für größere Drei-Zimmer-Apartments – jeweils innerhalb und außerhalb der Stadtzentren. Besonders bei den größeren Wohneinheiten fällt der Kostenunterschied signifikant aus, was für einen längeren Aufenthalt in tropischen Städten ein entscheidender Vorteil ist.

Auch beim **Kaufpreis von Immobilien** – gemessen am Preis pro Quadratmeter – zeigen sich klare Unterschiede. Tropische Länder liegen hier ebenfalls unter dem globalen Durchschnitt, was auf ein insgesamt geringeres Preisniveau auf den Wohnungsmärkten schließen lässt. Für Remote Worker mit langfristiger Perspektive oder Investmentinteresse könnte dies ein zusätzlicher Anreiz sein.

Ein Aspekt, der ebenfalls in der Grafik sichtbar wird, betrifft das **durchschnittliche Nettoeinkommen (x54)**. In tropischen Ländern fällt dieses im Vergleich zu globalen Werten **deutlich niedriger** aus. Im Kontext unseres Projekts stellt dies jedoch keinen Nachteil dar, da unser Einkommen unabhängig vom lokalen Arbeitsmarkt generiert wird. Vielmehr ergibt sich daraus ein **positiver Budgeteffekt**: Während die lokale Bevölkerung an ein niedrigeres Lohnniveau gebunden ist, profitieren Remote Worker von **günstigen Lebenshaltungskosten**, ohne dabei Einschränkungen beim Einkommen hinnehmen zu müssen.

### Fazit im Hinblick auf Remote Work

Die Auswertung bestätigt, dass tropische Länder im Bereich **Wohnen und Lebenshaltungskosten** besonders attraktiv sind. Die niedrigen Mieten und Immobilienpreise entlasten das monatliche Budget erheblich. Gleichzeitig wirkt sich das niedrige Einkommensniveau vor Ort nicht negativ auf die Lebensqualität von Remote Workern aus – im Gegenteil: Es schafft eine vorteilhafte wirtschaftliche Ausgangslage, um mit konstantem externen Einkommen einen vergleichsweise hohen Lebensstandard zu sichern.


In [ ]:
relevant_cols = ['x2','x8','x9','x10','x11','x12','x13','x14','x16','x17','x18','x19','x20','x21','x22','x23','x24']
tropical_avg = data_tropical[relevant_cols].mean()
global_avg = data[relevant_cols].mean()
diff = tropical_avg - global_avg
diff.plot(kind='bar', color='coral', figsize=(18, 6), edgecolor='black')
plt.axhline(0, color='black', linewidth=0.8)
plt.title('Differenz: Tropical Average - Global Average (Mietkosten, + Werte Tropische Länder haben mehr / - Werte Tropische Länder haben weniger)')
plt.xlabel('Kostenkategorie von Lokalen Essen')
plt.ylabel('Differenz')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Analyse der Lebensmittelpreise in tropischen Ländern

Ein zentraler Bestandteil unserer Analyse betrifft die lokalen Lebensmittelpreise, da eine gesunde und ausgewogene Ernährung für unseren geplanten Remote-Aufenthalt von besonderer Bedeutung ist. Die entsprechende Grafik zeigt die durchschnittliche Preisdifferenz tropischer Länder im Vergleich zum globalen Mittelwert für ausgewählte Grundnahrungsmittel.

Die Ergebnisse sind dabei insgesamt sehr deutlich: Die Mehrheit der untersuchten Lebensmittel ist in tropischen Ländern günstiger als im weltweiten Durchschnitt. Besonders ausgeprägt ist dieser Unterschied bei Produkten, die typischerweise lokal angebaut oder in großer Menge produziert werden – etwa bei **Bananen (x17)**. Hier lässt sich der Preisvorteil direkt auf die klimatischen Bedingungen und die regionale Landwirtschaft zurückführen.

Auch andere Grundnahrungsmittel – darunter **Äpfel (x16)**, **Tomaten (x19)**, **Kartoffeln (x20)** sowie verschiedene weitere Gemüse- und Obstsorten – zeigen signifikant niedrigere Preise. Diese Produkte bilden die Basis einer gesunden Ernährung und tragen wesentlich zur alltäglichen Lebensqualität bei.

Eine Ausnahme bildet die Variable **x24 (Bottle of Wine)**, deren Durchschnittskosten in tropischen Ländern leicht über dem globalen Vergleichswert liegen. Dieser Unterschied könnte auf Importabhängigkeit, Besteuerung oder eingeschränkte lokale Produktion zurückzuführen sein. Da Wein jedoch nicht zu den Grundnahrungsmitteln zählt und eine eher optionale Ausgabe darstellt, hat dieser Aspekt im Gesamtkontext nur geringe Relevanz.

### Fazit im Hinblick auf die Zielsetzung

Die Analyse der Lebensmittelpreise bestätigt, dass tropische Länder für unser geplantes Vorhaben besonders attraktiv sind. Die Kombination aus günstigen Mieten, niedrigeren Preisen für lokale Produkte und einem insgesamt erschwinglicheren Lebensstandard bildet eine ideale Grundlage für eine kosteneffiziente, gesundheitsbewusste Lebensweise vor Ort. Da unser Einkommen als Remote Worker unabhängig vom lokalen Arbeitsmarkt ist, profitieren wir direkt von diesen Preisvorteilen – insbesondere bei Lebensmitteln, die regelmäßig konsumiert werden und daher einen wesentlichen Teil der monatlichen Ausgaben ausmachen.


In [ ]:
relevant_cols = ['x29', 'x30', 'x31', 'x36','x38','x39','x40','x46']
tropical_avg = data_tropical[relevant_cols].mean()
global_avg = data[relevant_cols].mean()
diff = tropical_avg - global_avg
diff.plot(kind='bar', color='coral', figsize=(18, 6), edgecolor='black')
plt.axhline(0, color='black', linewidth=0.8)
plt.title('Differenz: Tropical Average - Global Average (Mietkosten, + Werte Tropische Länder haben mehr / - Werte Tropische Länder haben weniger)')
plt.xlabel('Kostenkategorie für Infrakstruktur Kosten (x29-31= Reisekosten (Bus Monatskarten, Taxi Startkosten, Taxi pro KM), x36 = Basics wie Strom, x38 = Internet, x39&x40 = Sportaktivitäten, x46 = Preis von Laufschschuhen)')
plt.ylabel('Differenz')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Analyse der Infrastruktur- und Freizeitkosten in tropischen Ländern

Für Remote Worker, die großen Wert auf einen gesunden und aktiven Lebensstil legen, ist es essenziell, neben klassischen Ausgabenkategorien wie Miete und Lebensmitteln auch die infrastrukturellen Grundkosten sowie die Möglichkeiten zur sportlichen Betätigung im Blick zu behalten. In tropischen Ländern, in denen kein eigenes Fahrzeug genutzt wird, spielen zusätzlich die Kosten für öffentliche Verkehrsmittel und Taxi eine zentrale Rolle – insbesondere über einen längeren Aufenthalt von sechs Monaten hinweg.

Die zugehörige Grafik zeigt die durchschnittlichen Kostenunterschiede zwischen tropischen Ländern und dem globalen Durchschnitt. Dabei ergeben sich folgende zentrale Erkenntnisse:

Die Reisekosten – abgebildet durch die Indikatoren für ÖPNV (x29) sowie Taxinutzung (x30, x31) – sind in tropischen Ländern insgesamt günstiger. Besonders deutlich fällt dies beim öffentlichen Nahverkehr auf, während die Taxipreise nur leicht unter dem globalen Niveau liegen. Gerade der ÖPNV stellt für Remote Worker eine kosteneffiziente und praktikable Lösung dar und wird über einen längeren Zeitraum hinweg zu einer relevanten Budgetgröße. Die günstigen Preise in diesem Bereich sind daher ein klarer Pluspunkt.

Auch bei den infrastrukturellen Fixkosten – etwa Strom, Wasser, Heizung und Müllentsorgung (x36) – schneiden tropische Länder positiv ab. Diese Ausgaben sind im Durchschnitt signifikant niedriger als global, was das monatliche Budget nachhaltig entlastet.

Die Kosten für Internetanschluss (x38) liegen hingegen leicht über dem globalen Durchschnitt. Für Remote Work ist ein stabiler und leistungsfähiger Internetzugang jedoch unerlässlich. Der leichte Aufpreis ist deshalb akzeptabel – insbesondere, da er durch Einsparungen in anderen Bereichen (z. B. Wohnen, Lebensmittel) kompensiert wird.

Ein weiterer wichtiger Aspekt betrifft die Möglichkeiten zur sportlichen Betätigung. Die Preise für Fitnessstudio-Mitgliedschaften (x39) und Tennisplätze (x40) sind in tropischen Ländern im Durchschnitt moderat bis leicht günstiger. Diese günstigen Zugangsmöglichkeiten fördern einen aktiven Lebensstil vor Ort und tragen positiv zur Lebensqualität bei. Auch bei der Sportausrüstung – konkret bei Laufschuhen (x46) – lassen sich Unterschiede feststellen: Die Preise liegen unter dem globalen Durchschnitt, was möglicherweise auf lokale Produktionsbedingungen oder geringere Importkosten in einigen tropischen Regionen zurückzuführen ist.

### Fazit für die Standortwahl

Insgesamt lässt sich festhalten, dass tropische Länder auch im Bereich Infrastruktur und Freizeit überzeugende Argumente liefern. Die Fixkosten für Energie, Mobilität und Sport fallen überwiegend günstiger aus als im globalen Mittel. Zwar sind die Internetkosten leicht erhöht, dieser Umstand ist jedoch im Gesamtbild vernachlässigbar. Die Ergebnisse dieser Analyse unterstreichen, dass tropische Länder nicht nur wirtschaftlich sinnvoll, sondern auch im Hinblick auf Lebensqualität und Alltagsgestaltung attraktive Optionen für Remote Work darstellen.


### Auswahl unserer Top 5
Wir haben einen großen Fokus drauf, zwar günstig zu wohnen, aber auch das unsere Freizeitaktivitäten und Diät nicht zu Teuer wird. Daher werden die Gewichtungen wie folgt gesetzt:

Wohnen: 0.8x

Internet und Strom: 1x

Nahrung zum Selber Kochen: 0.85x

Restaurants: 0.6x

Freizeit und Reisen: 0.9x

Klamotten: 0.5x

In [ ]:

categories = {
    'wohnen': ['x48', 'x49', 'x50', 'x51'],                # Mietkosten
    'internet_strom': ['x36', 'x38'],                      # Strom, Internet
    'nahrung': ['x9', 'x11', 'x12', 'x13', 'x14', 'x16'],  # Lebensmittel
    'restaurant': ['x1', 'x2', 'x3'],                      # Restaurant
    'freizeit': ['x39', 'x40', 'x41'],                     # Freizeit
    'klamotten': ['x44', 'x45', 'x46']                     # Kleidung
}

weights = {
    'wohnen': 0.8,
    'internet_strom': 1.0,
    'nahrung': 0.85,
    'restaurant': 0.6,
    'freizeit': 0.9,
    'klamotten': 0.5
}

# Init: lege für jede Kategorie einen Score an
for cat in categories.keys():
    data_tropical[f'score_{cat}'] = 0

# Punkte vergeben
for cat, cols in categories.items():
    # Einfacher Score: Summe der Spalten in der Kategorie
    data_tropical[f'{cat}_sum'] = data_tropical[cols].sum(axis=1)
    
    # Top 5 mit Punktvergabe
    top5 = data_tropical.nsmallest(5, f'{cat}_sum')  # falls "billiger = besser"
    data_tropical.loc[top5.index, f'score_{cat}'] = [5,4,3,2,1]  # automatisch von 5 bis 1 vergeben

# Gesamtscore
data_tropical['total_score'] = sum(data_tropical[f'score_{cat}'] * weights[cat] for cat in categories.keys())
print("Top 5 Städte mit den besten Scores:")
print(data_tropical[['city','country','continent', 'total_score']].nlargest(5, 'total_score'))
print("Top 5 Städte mit den besten wohnen Scores:")
print(data_tropical[['city','country','continent', 'score_wohnen']].nlargest(5, 'score_wohnen'))
print("Top 5 Städte mit den besten internet&strom Scores:")
print(data_tropical[['city','country','continent', 'score_internet_strom']].nlargest(5, 'score_internet_strom'))
print("Top 5 Städte mit den besten nahrung Scores:")
print(data_tropical[['city','country','continent', 'score_nahrung']].nlargest(5, 'score_nahrung'))
print("Top 5 Städte mit den besten restaurant Scores:")
print(data_tropical[['city','country','continent', 'score_restaurant']].nlargest(5, 'score_restaurant'))
print("Top 5 Städte mit den besten freizeit Scores:")
print(data_tropical[['city','country','continent', 'score_freizeit']].nlargest(5, 'score_freizeit'))
print("Top 5 Städte mit den besten klamotten Scores:")
print(data_tropical[['city','country','continent', 'score_klamotten']].nlargest(5, 'score_klamotten'))

### Interpretation der Scoring-Ergebnisse

#### Gesamtbewertung der Städte

Die umfassende Bewertung der Städte auf Basis verschiedener Kostenkategorien zeigt ein eindeutiges Bild: Die höchsten Gesamtscores wurden von Städten in **Südasien** (Colombo), **Lateinamerika** (Bucaramanga, Pereira) und **Südostasien** (Medan) erzielt. Diese Städte vereinen günstige Lebenshaltungskosten mit solider Infrastruktur und attraktiven Freizeitmöglichkeiten – alles zentrale Kriterien für Remote Work. Besonders hervorzuheben ist **Colombo (Sri Lanka)**, das sich in mehreren Kategorien an der Spitze positionieren konnte und insgesamt den höchsten Gesamtscore erreicht. **Kolumbien** ist ebenfalls stark vertreten und bietet mit Bucaramanga und Pereira gleich zwei Städte in den Top 5, was auf insgesamt günstige regionale Bedingungen schließen lässt.

---

### Wohnkosten

Die Kategorie Wohnen ist für Remote Worker von zentraler Bedeutung, da Mietkosten oft den größten Ausgabenposten darstellen. Hier schneiden insbesondere **Medan** sowie mehrere kolumbianische Städte wie Bucaramanga und Pereira hervorragend ab. Die Analyse zeigt, dass in diesen Städten sowohl Mieten im Zentrum als auch außerhalb vergleichsweise niedrig sind, was sie für einen langfristigen Aufenthalt besonders attraktiv macht.

---

### Infrastruktur: Internet & Strom

Ein weiteres zentrales Kriterium für ortsunabhängiges Arbeiten ist eine stabile und bezahlbare Versorgung mit Strom und Internet. In dieser Kategorie überzeugt insbesondere **Colombo**, das den besten Score erreicht. Auch **Accra** (Ghana) und **Shah Alam** (Malaysia) schneiden positiv ab. Für digitale Arbeitsmodelle ist diese infrastrukturelle Qualität essenziell und trägt wesentlich zur Standortwahl bei.

---

### Selbstversorgung: Lebensmittelpreise

Die Auswertung zeigt, dass vor allem Städte in **Kolumbien** in der Kategorie Selbstversorgung besonders stark abschneiden. Städte wie Manizales, Bucaramanga und Pereira punkten durch niedrige Preise für Grundnahrungsmittel, Obst und Fleisch. Dies macht eine gesunde und kostengünstige Ernährung vor Ort möglich – ein wichtiger Aspekt für Personen mit gesundheitsbewusstem Lebensstil.

---

### Restaurantpreise

Neben der Selbstversorgung spielen auch Restaurantkosten eine Rolle für die Lebensqualität. Hier dominieren Städte in **Indonesien** (Yogyakarta, Medan) sowie **Colombo**. Sie bieten günstige Optionen für regelmäßige Restaurantbesuche, was die kulturelle Erfahrung vor Ort und das soziale Leben fördern kann, ohne das Budget stark zu belasten.

---

### Freizeitkosten

Auch Freizeitangebote sind für die Attraktivität eines Aufenthaltsorts entscheidend. Die besten Werte in dieser Kategorie erzielten erneut **Colombo**, **Pereira** und **Bucaramanga**. Diese Städte bieten günstige Möglichkeiten für Fitness, Sport und Kino – Aktivitäten, die zur Aufrechterhaltung eines gesunden, aktiven Lebensstils beitragen.

---

### Kleidungskosten

Die Kategorie Kleidung hat eine geringere Gewichtung im Gesamtscore, liefert jedoch ergänzende Informationen. Hier fallen vor allem afrikanische Städte wie **Accra** und **Hargeysa** positiv auf. Aufgrund der geringen Bedeutung für den Alltag wurde dieser Faktor jedoch lediglich mit dem Faktor 0.5 berücksichtigt und hat daher weniger Einfluss auf die Endauswahl.

---
 
### Gesamtfazit aller Plots

Die Ergebnisse aller durchgeführten Analysen und Visualisierungen zeigen ein konsistentes Bild: Tropisch warme Länder bieten aus ökonomischer Sicht **ideale Voraussetzungen für Remote Work**. Die Daten belegen:

- **Signifikant niedrigere Lebenshaltungskosten** in zentralen Ausgabenkategorien wie Wohnen, Ernährung, Infrastruktur und Mobilität.
- **Günstige Bedingungen für eine gesundheitsbewusste Lebensweise**, etwa durch niedrigere Preise für Sporteinrichtungen, frische Lebensmittel und Freizeitaktivitäten.
- **Regionale Vorteile** vor allem in Asien, Afrika und Lateinamerika, wobei Städte in Sri Lanka, Kolumbien und Indonesien besonders häufig mit positiven Werten hervorstechen.
- **Infrastruktur- und Mobilitätskosten** sind ebenfalls in tropischen Regionen im Mittel günstiger, wodurch sich zusätzliche Einsparpotenziale ergeben.
- **Das Einkommen als Remote Worker** wird in tropischen Regionen besonders effizient eingesetzt, da lokale Preise deutlich unter dem Niveau westlicher Industriestaaten liegen.

Insgesamt bestätigen die Plots und Analysen die anfangs formulierten Hypothesen in weiten Teilen und liefern eine solide Grundlage für die Auswahl geeigneter tropischer Städte. Die identifizierten Regionen bieten nicht nur ein angenehmes Klima, sondern auch **eine hohe Lebensqualität bei gleichzeitig überschaubarem finanziellen Aufwand** – ideale Voraussetzungen für ein produktives und gesundes Arbeiten im Ausland.

---  
### Zusammenfassendes Fazit

Insgesamt zeigt die Analyse ein vielfältiges Bild attraktiver Remote-Work-Städte, wobei insbesondere Regionen in **Asien**, **Lateinamerika** und **Afrika** hervorstechen. Besonders auffällig ist die Stärke **Kolumbiens**, das in mehreren Kategorien durchgehend gute Bewertungen erzielt. Auch **Colombo** überzeugt durchgängig – es bietet eine sehr gute Kombination aus günstiger Infrastruktur, moderaten Lebenshaltungskosten und attraktiven Freizeitmöglichkeiten. Als afrikanische Option stellt **Accra** eine interessante Alternative dar, insbesondere durch seine solide Infrastruktur und günstige Kleidungskosten.

Trotz der Vielfalt der Top-Platzierungen ergibt sich ein klarer Gesamtsieger: **Colombo**. Die Stadt vereint alle für Remote Worker entscheidenden Faktoren und überzeugt sowohl in der Einzelwertung als auch im Gesamtranking. Damit ist sie aus Sicht dieses Projekts die vielversprechendste Wahl für einen sechsmonatigen Aufenthalt.

## Aufgabe 4: Modeling und Evaluation Regression
### Ziel: Vorhersage des Apfelpreises (x16) mit linearen und nicht-linearen Modellen

#### Datenvorbereitung

**Zweck:**  
Die Datenvorbereitung zielt darauf ab, eine aussagekräftige Basis für die Regressionsmodelle zu schaffen, indem wir die Zielvariable und die relevantesten erklärenden Variablen auswählen und die Daten für die Modellierung optimieren.

**Auswahl der Zielvariable und Features:**  
- **Zielvariable:** `x16` (Apfelpreis)  
  Diese Variable wurde ausgewählt, da sie das zentrale Vorhersageziel des Modells darstellt.
- **Features:**  
  Basierend auf der Korrelationsanalyse wurden die Variablen ausgewählt, die die höchste statistische Beziehung zum Apfelpreis aufweisen. Hierzu zählen:  
  - `x11` (Reis)  
  - `x10` (Brot)  
  - `x54` (Gehalt)  
  Zusätzlich fließt der Faktor **Kontinent** mit ein, um regionale Unterschiede abzubilden.

**Bereinigung und Kodierung:**  
- **Datenbereinigung:**  
  Es werden alle Datensätze mit fehlenden Werten entfernt, um sicherzustellen, dass das Modell auf konsistenten und vollständigen Informationen trainiert wird.
- **One-Hot-Encoding:**  
  Der kategoriale Faktor „Kontinent“ wird mittels One-Hot-Encoding transformiert, wobei aufgrund der Multikollinearität ein Dummy (drop_first=True) entfernt wird. Dies ermöglicht die Einbeziehung von Kontinenten als zusätzliche Einflussgröße ohne zusätzliche Modellverzerrung.

**Begründung:**  
Die Kombination der am stärksten korrelierten Features mit dem Kontinent als geographischem Indikator soll sicherstellen, dass sowohl ökonomische als auch regionale Faktoren angemessen berücksichtigt werden. Dies ermöglicht eine differenzierte Analyse der Einflüsse auf den Apfelpreis und gewährleistet, dass das anschließende Modell robust und interpretierbar ist.


In [ ]:
# Zielvariable & Features definieren
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

target = 'x16'
features = ['x11', 'x10', 'x54', 'continent']  # Reis, Brot, Gehalt, Kontinent

# Daten bereinigen und kodieren
data_reg = data[features + [target]].dropna()
data_reg = pd.get_dummies(data_reg, columns=['continent'], drop_first=True)

# Ausgabe der Form und ersten Zeilen
print("Shape von data_reg:", data_reg.shape)
print("\nErste 5 Zeilen von data_reg:")
print(data_reg.head())

#### Korrelationsanalyse

**Ergebnis:**  
Die Analyse zeigt, dass die Variable `x10` (Brot) mit einem Korrelationswert von 0,76 den stärksten Zusammenhang mit dem Apfelpreis (`x16`) aufweist.

**Interpretation:**  
Ein hoher positiver Korrelationskoeffizient deutet darauf hin, dass steigende Brotpreise häufig mit höheren Preisen im Bereich der Lebensmittel einhergehen. Somit kann der Brotpreis als ein aussagekräftiger Indikator für generelle Trends in der Lebensmittelteuerung betrachtet werden.

**Handlungsempfehlung:**  
Auf Grundlage dieser Erkenntnisse empfiehlt es sich, bei der linearen Regression neben dem Apfelpreis als Zielvariable die Features für Reis, Brot und Gehalt zu verwenden, da diese Variablen die zentrale Preisentwicklung im Lebensmittelsektor widerspiegeln.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Korrelationen für die Zielvariable (x16, Apfelpreis) abrufen und nach Stärke sortieren
corr_values = data_reg.corr()['x16'].sort_values(ascending=False)

# Ausgabe der berechneten Korrelationswerte
print("Korrelationswerte mit x16:")
print(corr_values)

# Erstelle den Barplot
plt.figure(figsize=(8, 6))
ax = sns.barplot(
    x=corr_values.values, 
    y=corr_values.index, 
    hue=corr_values.index, 
    palette="coolwarm",
    dodge=False 
)
plt.legend([], [], frameon=False)
plt.title('Korrelationen mit Apfelpreis (x16)')
plt.xlabel('Korrelationswert')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


#### Datenaufteilung

**Vorgehen:**  
Die Daten werden im Verhältnis 70 % Training, 15 % Validierung und 15 % Test aufgeteilt.

**Begründung:**  
Die Aufteilung sorgt für eine ausreichende Datenmenge im Training und erlaubt zugleich eine zuverlässige Überprüfung auf Overfitting über die Validierungsdaten. Die Testdaten werden ausschließlich zur abschließenden Bewertung des Modells verwendet.  
Ein fester `random_state` stellt sicher, dass die Aufteilung reproduzierbar bleibt.

**Verteilung der Zielvariable:**  
Es werden drei Histogramme (je ein Plot für y_train, y_val und y_test) erstellt, die die Verteilung des Apfelpreises (x16) in den jeweiligen Datensätzen zeigen. Zusätzlich wird jeweils eine Dichtekurve (Kernel Density Estimate, KDE) eingezeichnet. So kann überprüft werden, ob die Verteilungen in allen Datensätzen ähnlich sind und damit ein konsistentes Sampling vorliegt.


In [ ]:
# Zielvariable und Features separieren
X = data_reg.drop(columns=[target])   # Alle erklärenden Variablen
y = data_reg[target]                    # Zielvariable (Apfelpreis)

# Aufteilung in Trainings- und temporären Datensatz (70 % Training, 30 % temporär)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Aufteilung des temporären Datensatzes in Validierungs- und Testdaten (jeweils 15 %)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Prüfung der Formen der Datensätze ausgeben
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

# Visualisierung: Verteilung der Zielvariable in den Split-Sets
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
sns.histplot(y_train, kde=True, color='skyblue')
plt.title("Distribution of y_train")
plt.xlabel("Apfelpreis")
plt.ylabel("Count")

plt.subplot(1,3,2)
sns.histplot(y_val, kde=True, color='salmon')
plt.title("Distribution of y_val")
plt.xlabel("Apfelpreis")
plt.ylabel("Count")

plt.subplot(1,3,3)
sns.histplot(y_test, kde=True, color='limegreen')
plt.title("Distribution of y_test")
plt.xlabel("Apfelpreis")
plt.ylabel("Count")

plt.tight_layout()
plt.show()


### Lineare Regression

**Modellwahl:**  
Für den ersten Modellansatz wird eine einfache lineare Regression mit drei erklärenden Variablen (Reis, Brot, Gehalt) verwendet. Diese Kombination wurde auf Basis der Korrelationsanalyse gewählt.

**Begründung:**  
Die lineare Regression eignet sich gut für einen ersten, interpretierten Modellansatz. Sie ermöglicht eine direkte Ableitung der Gewichtung einzelner Einflussfaktoren und ist rechnerisch effizient. Die Reduktion auf drei numerische Features erhöht die Nachvollziehbarkeit der Ergebnisse.

**Bewertung:**  
Zur Modellbewertung werden zwei etablierte Metriken verwendet:  
- **R²** gibt an, wie viel Varianz im Zielwert durch das Modell erklärt wird.  
- **RMSE** misst die durchschnittliche Abweichung der Vorhersage in USD.

**Ergebnis:**  
Das Modell erreicht einen R²-Wert von 0,67 auf den Trainingsdaten, was heißt, dass etwa 2/3 der Varianz erklärt werden können. Der RMSE ist mit 0.71 USD etwas hoch und es gibt noch Luft nach oben.
**Visualisierung:**  
Zur besseren Beurteilung des Modells wurden zwei Plots erstellt:

- **Scatterplot (Vorhergesagte vs. Tatsächliche Werte):**  
  Zeigt, wie gut das Modell die Apfelpreise trifft. Punkte entlang der roten Diagonale stehen für genaue Vorhersagen.

- **Residuenplot:**  
  Stellt dar, wie die Abweichungen (Residuen) vom Modell verteilt sind. Eine zufällige Streuung um die Null-Linie deutet darauf hin, dass das Modell keine systematischen Fehler macht.


In [ ]:
# Konfiguration
SELECTED_FEATURES = ['x11', 'x10', 'x54']  # Reis, Brot, Gehalt
PLOT_STYLE = {
    'train': {'color': 'blue', 'label': 'Train'},
    'val': {'color': 'green', 'label': 'Validation'}
}

# Modelltraining und Vorhersagen
model = LinearRegression()
model.fit(X_train[SELECTED_FEATURES], y_train)

y_preds = {
    'train': model.predict(X_train[SELECTED_FEATURES]),
    'val': model.predict(X_val[SELECTED_FEATURES])
}

# Metrikenberechnung und Ausgabe
print(f"Train R²: {r2_score(y_train, y_preds['train']):.2f}")
print(f"Val RMSE: {mean_squared_error(y_val, y_preds['val']):.2f} USD")

# Visualisierungsfunktionen
def plot_actual_vs_predicted(ax, y_true, y_pred, context):
    sns.scatterplot(x=y_true, y=y_pred, ax=ax, alpha=0.6, color=context['color'])
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    ax.set_title(f"{context['label']}: Vorhergesagte vs. tatsächliche Werte")
    ax.set_xlabel("Tatsächliche Werte")
    ax.set_ylabel("Vorhergesagte Werte")

# Gesamtvisualisierung
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Reihe: Vorhersageplots
plot_actual_vs_predicted(axes[0, 0], y_train, y_preds['train'], PLOT_STYLE['train'])
plot_actual_vs_predicted(axes[0, 1], y_val, y_preds['val'], PLOT_STYLE['val'])

# 2. Reihe: Residuenanalyse
residuals = y_val - y_preds['val']

# Residuenplot
sns.scatterplot(x=y_preds['val'], y=residuals, ax=axes[1, 0], alpha=0.6, color='purple')
axes[1, 0].axhline(0, color='red', linestyle='--')
axes[1, 0].set_title("Residuen vs. Vorhersage (Validation)")
axes[1, 0].set_xlabel("Vorhersage")
axes[1, 0].set_ylabel("Residuen")

# Residuenverteilung
sns.histplot(residuals, kde=True, ax=axes[1, 1], color='orange', bins=30)
axes[1, 1].set_title("Verteilung der Residuen")
axes[1, 1].set_xlabel("Residuen")
axes[1, 1].set_ylabel("Anzahl")

plt.tight_layout()
plt.show()

### Random Forest Regression

**Modellwahl:**  
Random Forest wird als nicht-lineares Ensemble-Verfahren eingesetzt, um die Vorhersageleistung im Vergleich zur linearen Regression zu verbessern. Das Modell nutzt Entscheidungsbäume und aggregiert deren Ergebnisse, wodurch auch komplexe Zusammenhänge und Interaktionen zwischen Variablen erfasst werden können.

**Begründung:**  
Im Gegensatz zur linearen Regression ist Random Forest in der Lage, nicht-lineare Effekte und wechselseitige Abhängigkeiten zwischen Features zu modellieren. Das Verfahren ist robust gegenüber Ausreißern und skaliert gut mit einer größeren Anzahl an Merkmalen.

**Bewertung:**  
- **R²** auf den Validierungsdaten liegt bei ca. 0,72 und zeigt eine etwas verbesserte Modellqualität.  
- **RMSE** fällt im Vergleich zur linearen Regression geringer aus, was auf präzisere Vorhersagen hinweist.

### Visualisierung der Modellergebnisse

Zur Bewertung der Vorhersagequalität und zur Analyse des Modellverhaltens wurden folgende Plots erstellt:

#### Vorhergesagte vs. tatsächliche Werte  
Ein Scatterplot zeigt, wie gut die vom Modell vorhergesagten Apfelpreise mit den tatsächlichen Werten übereinstimmen. Punkte, die nah an der Diagonalen liegen, deuten auf präzise Vorhersagen hin.

### Residuenanalyse

**Ergebnis:**  
Die Residuen sind weitgehend zufällig um die Nulllinie verteilt, was auf ein gut angepasstes Modell hinweist.

**Interpretation:**  
Die symmetrische Verteilung der Residuen spricht gegen systematische Fehler – also dafür, dass das Modell keine strukturelle Verzerrung in bestimmten Preisbereichen aufweist. Eine leichte Rechtsschiefe deutet allerdings auf einzelne Ausreißer mit besonders hohen Apfelpreisen hin, wie sie typischerweise in sehr teuren Städten auftreten.

**Hinweis:**  
Solche Ausreißer können die Modellqualität beeinflussen, insbesondere bei Fehlermaßen wie dem RMSE. Dennoch bleibt die Gesamtverteilung stabil und bestätigt die Zuverlässigkeit des Modells.
#### Feature Importance  
Die Feature Importance zeigt, welche Variablen vom Random Forest Modell als besonders einflussreich eingeschätzt wurden. So lassen sich indirekt Rückschlüsse auf die treibenden Faktoren der Preisbildung ziehen – z. B. ist das Durchschnittsgehalt (x54) der wichtigste Prädiktor.

#### Histogramm der Residuen  
Die Verteilung der Residuen hilft zu beurteilen, ob das Modell im Mittel systematisch über- oder unterschätzt. Eine symmetrische Verteilung um null spricht für ein ausgewogenes Modell.

Diese Visualisierungen liefern gemeinsam ein umfassendes Bild über die Qualität, Genauigkeit und Eigenschaften des trainierten Random Forest Modells.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Modelltraining
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Vorhersagen und Metriken
y_pred_rf_train = model_rf.predict(X_train)  # Für Overfitting-Check
y_pred_rf_val = model_rf.predict(X_val)

print(f"Train R²: {r2_score(y_train, y_pred_rf_train):.2f}")
print(f"Val R²: {r2_score(y_val, y_pred_rf_val):.2f}")
print(f"Val RMSE: {mean_squared_error(y_val, y_pred_rf_val):.2f} USD")

# 1. Actual vs. Predicted mit Train/Val-Vergleich
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Training
sns.scatterplot(x=y_train, y=y_pred_rf_train, ax=ax[0], alpha=0.6, color='blue')
ax[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')
ax[0].set_title(f"Training (R²={r2_score(y_train, y_pred_rf_train):.2f})")
ax[0].set_xlabel('Actual')
ax[0].set_ylabel('Predicted')

# Validation
sns.scatterplot(x=y_val, y=y_pred_rf_val, ax=ax[1], alpha=0.6, color='green')
ax[1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--')
ax[1].set_title(f"Validation (R²={r2_score(y_val, y_pred_rf_val):.2f})")
ax[1].set_xlabel('Actual')

plt.tight_layout()
plt.show()

# 2. Feature Importance
feature_imp = pd.Series(model_rf.feature_importances_, 
                        index=X_train.columns).sort_values(ascending=False)[:10]

# 3. Residual Analysis
residuals = y_val - y_pred_rf_val

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Residuals vs Predicted
sns.scatterplot(x=y_pred_rf_val, y=residuals, ax=ax[0], alpha=0.6, color='purple')
ax[0].axhline(0, color='red', linestyle='--')
ax[0].set_title('Residuals vs Predicted Values')
ax[0].set_xlabel('Predicted Values')
ax[0].set_ylabel('Residuals')

# Residual Distribution
sns.histplot(residuals, kde=True, ax=ax[1], bins=30, color='orange')
ax[1].set_title('Residual Distribution')
ax[1].set_xlabel('Residuals')
plt.tight_layout()
plt.show()

# Aufgabe 5


In [ ]:
# Feature Auswahl
relevant_cols = [
    # 🍽️ Restaurant
    'x1',  # Meal, Inexpensive Restaurant
    'x2',  # Meal for 2 People, Mid-range Restaurant
    'x3',  # McMeal at McDonalds

    # 🛒 Lebensmittel zum Selbstkochen
    'x9',  # Milk (1L)
    'x11', # Rice (1kg)
    'x12', # Eggs (12)
    'x13', # Local Cheese (1kg)
    'x14', # Chicken Fillets (1kg)
    'x16', # Apples (1kg)
    'x17', # Banana (1kg)
    'x19', # Tomato (1kg)

    # 🏠 Wohnen
    'x50', # Apartment (3 bedrooms) in City Centre
    'x51', # Apartment (3 bedrooms) Outside of Centre

    # 🔌 Infrastruktur
    'x36', # Basic utilities (Electricity, Heating, Water, etc.)
    'x38', # Internet

    # 📶 Kommunikation (optional)
    'x37', # Prepaid Mobile Tariff

    # 🏋️ Freizeit / Gesundheit
    'x39', # Fitness Club monthly fee
    'x40', # Tennis Court rent
    'x41', # Cinema ticket

    # 🚌 Mobilität
    'x28', # One-way Ticket (Local Transport)
    'x29', # Monthly Pass
]


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Beispiel: nur numerische Spalten nehmen

# Standardisieren
scaler = StandardScaler()

sse = []
sil_scores = []
X = scaler.fit_transform(data_tropical[relevant_cols])  

for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    sse.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X, labels))

# Plot
plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
plt.plot(range(2,10), sse, marker='o')
plt.title('Elbow-Methode')
plt.xlabel('Anzahl Cluster')
plt.ylabel('SSE')

plt.subplot(1,2,2)
plt.plot(range(2,10), sil_scores, marker='o', color='green')
plt.title('Silhouette Score')
plt.xlabel('Anzahl Cluster')
plt.ylabel('Score')
plt.tight_layout()
plt.show()

sse = []
sil_scores = []
X = scaler.fit_transform(data[relevant_cols])  

for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    sse.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X, labels))

# Plot
plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
plt.plot(range(2,10), sse, marker='o')
plt.title('Elbow-Methode')
plt.xlabel('Anzahl Cluster')
plt.ylabel('SSE')

plt.subplot(1,2,2)
plt.plot(range(2,10), sil_scores, marker='o', color='green')
plt.title('Silhouette Score')
plt.xlabel('Anzahl Cluster')
plt.ylabel('Score')
plt.tight_layout()
plt.show()


### Wahl der Clusteranzahl

Zur Bestimmung einer geeigneten Anzahl an Clustern wurden zwei gängige Evaluationsmethoden aus dem Bereich des unüberwachten Lernens verwendet: die **Elbow-Methode** sowie der **Silhouette-Score**. Beide Verfahren liefern Hinweise darauf, wie viele Cluster eine sinnvolle und interpretierbare Gruppierung der Daten ermöglichen.

Die **Elbow-Methode** visualisiert die sogenannte "Sum of Squared Errors" (SSE), also die quadratische Abweichung der Datenpunkte von ihren jeweiligen Clusterzentren. Der Punkt, an dem die Kurve deutlich abflacht, wird als optimaler „Knick“ (Elbow) interpretiert, da zusätzliche Cluster danach nur noch geringe Verbesserungen liefern.

Der **Silhouette-Score** bewertet die Qualität der Clusterbildung basierend auf der durchschnittlichen Distanz eines Punktes zu seinem eigenen und den benachbarten Clustern. Ein hoher Wert deutet auf kompakte und gut separierte Cluster hin.

#### Ergebnisse für die tropischen Daten

Wie im oberen Teil des Screenshots ersichtlich, deutet die Elbow-Kurve bei **k = 4** auf einen sinnvollen Knick hin. Der Silhouette-Score ist zwar bei **k = 2** am höchsten, nimmt jedoch erst ab **k = 5** signifikant ab. Die moderate Abflachung zwischen **k = 3** und **k = 4** spricht dafür, dass in diesem Bereich eine gute Balance zwischen Modellgüte und Interpretierbarkeit liegt.

Daher wurde für den tropischen Datensatz **k = 4** gewählt. Diese Anzahl erlaubt es, verschiedene Typen tropischer Städte – z. B. günstig mit einfacher Infrastruktur versus etwas teurer mit besserer Ausstattung – sinnvoll voneinander zu unterscheiden.

#### Ergebnisse für die globalen Daten

Auch im unteren Bereich des Screenshots, der die globale Stichprobe darstellt, zeigt sich ein deutlicher Knick der Elbow-Kurve bei **k = 4**. Gleichzeitig fällt der Silhouette-Score nach **k = 4** spürbar ab, während er bei **k = 3** und **k = 4** noch auf einem vergleichsweise stabilen Niveau verbleibt.

Daher wurde auch für den globalen Datensatz **k = 4** als sinnvolle Clusteranzahl gewählt. Dies ermöglicht eine differenzierte Analyse von Stadttypen entlang der Achsen „Kosten“ und „Infrastruktur“, ohne die Cluster zu stark zu fragmentieren.

In [ ]:
X_tropical = scaler.fit_transform(data_tropical[relevant_cols])  

kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X_tropical)
cluster_means = data_tropical.copy()
cluster_means['cluster'] = kmeans.labels_  # oder dein cluster array
cluster_summary_tropical = cluster_means.groupby('cluster')[relevant_cols].mean()

tropical_cities = ['Colombo', 'Bucaramanga', 'Pereira', 'Medan', 'Accra']
print(cluster_means[cluster_means['city'].isin(tropical_cities)][['city', 'cluster']])

X = scaler.fit_transform(data[relevant_cols])  

kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X)

data['cluster'] = kmeans.labels_
cluster_summary_global = data.groupby('cluster')[relevant_cols].mean()
print(data[data['city'].isin(tropical_cities)][['city', 'cluster']])


plt.figure(figsize=(12,6))
plt.subplot(1,2,1)

sns.heatmap(cluster_summary_tropical.T, cmap='coolwarm', annot=True, fmt=".1f")
plt.title("Durchschnittliche Werte je Tropischen Cluster")
plt.xlabel("Cluster")
plt.ylabel("Merkmal")
plt.tight_layout()

plt.subplot(1,2,2)
sns.heatmap(cluster_summary_global.T, cmap='coolwarm', annot=True, fmt=".1f")
plt.title("Durchschnittliche Werte je Globalen Cluster")
plt.xlabel("Cluster")
plt.ylabel("Merkmal")
plt.tight_layout()
plt.show()


### Vergleich der Clusterstruktur: Tropische vs. globale Daten

Die beiden Heatmaps zeigen den Durchschnittswert ausgewählter Merkmale pro Cluster – getrennt nach einem Clustering basierend auf ausschließlich tropischen Städten (links) und auf allen verfügbaren Städten weltweit (rechts). Die Cluster wurden jeweils mit dem K-Means-Verfahren mit vier Gruppen berechnet.

#### Tropische Städte (links):

Im tropischen Datensatz lässt sich eine klar **aufsteigende Kostenstruktur von Cluster 0 bis Cluster 3** erkennen. Cluster 0 weist die niedrigsten Durchschnittswerte für nahezu alle Merkmale auf – darunter Lebensmittelpreise, Mietkosten (`x50`, `x51`) sowie Freizeit- und Infrastrukturkosten (`x36`, `x38`, `x39`). Cluster 3 hingegen enthält die höchsten Ausprägungen in nahezu allen betrachteten Bereichen.

Diese monotone Zunahme spricht dafür, dass die Cluster im tropischen Raum stark durch das allgemeine Preisniveau geprägt sind. Städte in Cluster 0 stellen damit besonders günstige, einfache Lebensumfelder dar, während Cluster 3 vermutlich teurere Metropolen mit höherer Infrastruktur- und Lebenshaltungskosten abbildet.

#### Globale Daten (rechts):

Im globalen Datensatz zeigt sich ein differenzierteres Bild: Hier ist **Cluster 3 der günstigste**, während Cluster 0 und Cluster 2 deutlich höhere Werte aufweisen – insbesondere bei den Mietkosten (`x50`, `x51`) und bei Mobilitäts- sowie Freizeitpreisen (`x36`, `x39`). Cluster 1 positioniert sich dabei zwischen den Extremen und lässt sich als mittleres Preisniveau interpretieren.

Diese Struktur legt nahe, dass das globale Clustering nicht nur auf steigenden Kosten basiert, sondern auch **strukturelle Unterschiede zwischen Regionen und Kontinenten** abbildet – z. B. zwischen tropischen Städten mit niedriger Kaufkraft und westlichen Metropolen mit hohem Preisniveau.

#### Interpretation anhand der Top-5-Städte:

Die fünf zuvor identifizierten Favoritenstädte – Bucaramanga, Pereira, Accra, Medan und Colombo – befinden sich im tropischen Clustering alle in **Cluster 0**, also im günstigsten Segment. Im globalen Clustering sind dieselben Städte jedoch sämtlich in **Cluster 3** vertreten – dem günstigsten globalen Cluster. Diese Übereinstimmung bestätigt die Annahme, dass die gewählten Städte im internationalen Vergleich ein **sehr attraktives Preis-Leistungs-Verhältnis** aufweisen.

#### Fazit:

Der Vergleich beider Heatmaps zeigt deutlich, dass die **absolute Lage eines Clusters (z. B. „Cluster 3“) keine feste Bedeutung hat**, sondern im jeweiligen Kontext interpretiert werden muss. Im tropischen Kontext ist Cluster 3 teuer, im globalen hingegen besonders günstig. Für die Ergebnisinterpretation ist daher stets der Bezug zur zugrundeliegenden Stichprobe entscheidend.


In [ ]:
from sklearn.decomposition import PCA

# PCA durchführen
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X)

# PCA-Plot mit den bereits vorhandenen Clustern
plt.figure(figsize=(18, 10))
scatter = plt.scatter(
    pca_result[:, 0],
    pca_result[:, 1],
    c=data['cluster'],
    cmap='viridis',
    alpha=0.7,
    edgecolor='k'
)
plt.xlabel('PCA Komponente 1')
plt.ylabel('PCA Komponente 2')
plt.title('PCA-Visualisierung der Cluster (Globale Städte)')
plt.grid(True)
plt.colorbar(scatter, label='Cluster')
plt.tight_layout()
plt.show()

pca_result = pca.fit_transform(X_tropical)

# PCA-Plot mit den bereits vorhandenen Clustern
plt.figure(figsize=(18, 10))
scatter = plt.scatter(
    pca_result[:, 0],
    pca_result[:, 1],
    c=cluster_means['cluster'],
    cmap='viridis',
    alpha=0.7,
    edgecolor='k'
)
plt.xlabel('PCA Komponente 1')
plt.ylabel('PCA Komponente 2')
plt.title('PCA-Visualisierung der Cluster (Tropische Städte)')
plt.grid(True)
plt.colorbar(scatter, label='Cluster')
plt.tight_layout()
plt.show()

### Visualisierung der Cluster mithilfe von PCA

Um die Verteilung der durch das K-Means-Verfahren gebildeten Cluster zu untersuchen, wurde eine Hauptkomponentenanalyse (PCA) durchgeführt. Ziel war es, die mehrdimensionalen Daten auf zwei Dimensionen zu reduzieren und dadurch visuell interpretierbare Zusammenhänge zwischen den Clustern darzustellen.

---

#### Plot 1 – Globale Städte

Die erste Visualisierung zeigt die globale Clusterverteilung auf Basis aller Städte im Datensatz. Die Datenpunkte wurden mittels PCA auf zwei Hauptkomponenten projiziert und nach den jeweiligen Clustern eingefärbt. Es ist eine weitgehende Trennung der Cluster in verschiedenen Bereichen des Koordinatensystems erkennbar. Besonders auffällig ist die räumliche Konzentration einzelner Cluster, was auf eine gute Trennschärfe des K-Means-Verfahrens hinweist. Gleichzeitig zeigt sich eine gewisse Überlappung zwischen benachbarten Clustern (besonders Cluster 0 und 2, ebenfalls einflüsse in Cluster 3), was auf ähnliche Kostenstrukturen oder Mischformen zwischen den Lebenshaltungskosten hindeuten könnte.

---

#### Plot 2 – Tropische Städte

Die zweite Darstellung fokussiert ausschließlich auf die tropischen Städte. Auch hier wurden die Daten mittels PCA auf zwei Dimensionen reduziert. Die Clusterausprägungen erscheinen weniger differenziert als im globalen Vergleich. Die Mehrheit der tropischen Städte ist in zwei Hauptclustern zu finden, was mit den zuvor berechneten Verteilungen konsistent ist. Einzelne Ausreißer weisen jedoch auf Städte mit besonderen Kostenprofilen hin, die sich vom typischen Muster tropischer Regionen abheben.

---

### Interpretation

Die beiden Scatterplots unterstreichen die Aussagekraft der Clusteranalyse: Während globale Städte in vier gut voneinander trennbare Gruppen eingeteilt werden konnten, konzentrieren sich die tropischen Städte stärker in bestimmten Clustern. Dies stützt die These, dass tropische Regionen tendenziell homogenere Preisstrukturen aufweisen – vor allem im unteren bis mittleren Kostenniveau. Die PCA stellt somit eine sinnvolle Methode dar, um die durch Clustering gewonnenen Erkenntnisse visuell zu bestätigen und zu interpretieren.


In [ ]:
# 1. Cluster für tropisch/nicht-tropisch übernehmen (nicht neu fitten!)
data_tropical['cluster'] = data.loc[data_tropical.index, 'cluster']
data_non_tropical = data[~data.index.isin(data_tropical.index)].copy()
data_non_tropical['cluster'] = data.loc[data_non_tropical.index, 'cluster']

# 2. Verteilungen zählen
tropische_verteilung = data_tropical.groupby('cluster').size()
nicht_tropisch_verteilung = data_non_tropical.groupby('cluster').size()
gesamt = data.groupby('cluster').size()  # bereits global
print("Tropische Cluster:")
print(tropische_verteilung)
print("Nicht-tropische Cluster:")
print(nicht_tropisch_verteilung)
print("Gesamtverteilung:")
print(gesamt)

# 3. Prozentanteile berechnen
anteil_tropisch = (tropische_verteilung / gesamt * 100).fillna(0).round(1)
anteil_nicht_tropisch = (nicht_tropisch_verteilung / gesamt * 100).fillna(0).round(1)

# 4. Plot
plt.figure(figsize=(16, 8))
bars = plt.bar(anteil_tropisch.index, anteil_tropisch.values, color='mediumseagreen')
plt.title('Anteil tropischer Städte je Cluster')
plt.xlabel('Cluster')
plt.ylabel('Anteil tropischer Städte (%)')
plt.ylim(0, 100)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 2, f'{height:.1f}%', ha='center', fontsize=10)

plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


### Analyse der Clusterverteilung: Tropische vs. Nicht-tropische Städte

Die folgende Tabelle zeigt, wie viele Städte aus tropischen bzw. nicht-tropischen Regionen in jedem Cluster vertreten sind und welchen Anteil tropische Städte innerhalb des Clusters ausmachen:

| Cluster | Tropisch | Nicht-tropisch | Anteil tropisch (%) |
|---------|----------|----------------|----------------------|
| 0       | 1        | 48             | 2.0 %                |
| 1       | 6        | 307            | 1.9 %                |
| 2       | 5        | 176            | 2.8 %                |
| 3       | 66       | 314            | **17.4 %**           |

---

### Analyse der Clusterverteilung nach Herkunftsregion

Um die Zusammensetzung der ermittelten Cluster zu bewerten, wurde untersucht, wie sich tropische und nicht-tropische Städte über die vier gebildeten Cluster verteilen. Dabei zeigt sich eine deutlich ungleiche Verteilung zwischen den beiden Gruppen.

### Tropische Städte konzentrieren sich auf ein einziges Cluster

Die tropischen Städte sind nahezu ausschließlich im Cluster 3 vertreten. Von insgesamt 380 Städten in diesem Cluster stammen 66 aus tropischen Ländern, was einem Anteil von 17,4 % entspricht. In den anderen drei Clustern liegt der Anteil tropischer Städte bei lediglich 1,9 % bis 2,8 %. Diese starke Konzentration legt nahe, dass tropische Städte aus Sicht der Clusteranalyse strukturell sehr ähnlich sind und gemeinsam ein bestimmtes Profil aufweisen.

### Nicht-tropische Städte über alle Cluster hinweg verteilt

Im Gegensatz dazu sind die nicht-tropischen Städte wesentlich breiter verteilt. Alle drei verbleibenden Cluster (0, 1 und 2) werden fast ausschließlich von nicht-tropischen Städten dominiert. Besonders deutlich wird dies bei Cluster 1, das mit 307 nicht-tropischen Städten das größte Einzelcluster darstellt. Auch Cluster 2 enthält 176 dieser Städte, während Cluster 0 auf 48 kommt. Damit zeigt sich, dass nicht-tropische Städte sehr heterogen sind und sowohl günstige als auch hochpreisige Städte umfassen.

### Verbindung zur Hypothese 2

Die beobachtete Verteilung stützt die aufgestellte Hypothese 2:

> *Tropisch warme Länder haben tendenziell niedrigere Kosten als Europa und Nordamerika.*

Diese Aussage wird durch die Tatsache gestützt, dass sich tropische Städte im globalen Clustering fast ausschließlich in jenem Cluster wiederfinden, das – wie zuvor anhand der Cluster-Heatmaps festgestellt – die **niedrigsten durchschnittlichen Lebenshaltungskosten** aufweist. Cluster 3 ist in fast allen Kostenkategorien deutlich günstiger als die übrigen Gruppen und bildet somit ein klares Gegengewicht zu Clustern wie 0 oder 1, in denen westliche Städte mit hohem Preisniveau dominieren.

Darüber hinaus verdeutlicht die enge Gruppierung tropischer Städte, dass diese Länder neben einem insgesamt niedrigeren Preisniveau auch eine gewisse Homogenität in ihrer Kostenstruktur aufweisen. Die hohe Streuung nicht-tropischer Städte über sämtliche Cluster zeigt dagegen, dass westliche Städte deutlich differenzierter ausfallen und sich in ihrer wirtschaftlichen Struktur und Infrastrukturstärke stärker voneinander unterscheiden.

### Schlussfolgerung

Insgesamt unterstreicht die Clusteranalyse den Zusammenhang zwischen geographischer Lage und wirtschaftlichem Umfeld. Tropische Städte bilden eine relativ homogene, kostengünstige Gruppe innerhalb des globalen Clustermodells, während nicht-tropische Städte breiter gestreut sind und sowohl günstige als auch hochpreisige Cluster dominieren. Die Hypothese, dass tropisch warme Länder tendenziell mit niedrigeren Lebenshaltungskosten einhergehen, lässt sich auf Basis dieser Ergebnisse klar bestätigen.



## Aufg 6: Deployment: 
### Anleitung für Remote Worker: Günstige & exotische Städte für 6 Monate Workation  

## Ziel

Diese Analyse unterstützt Remote Worker aus der Kreativbranche bei der Auswahl geeigneter tropisch-warmer Städte für einen sechsmonatigen Aufenthalt. Bewertet wurden zentrale Kostenkategorien wie Wohnen, Infrastruktur, Ernährung und Freizeitangebote, mit Fokus auf Lebensqualität bei gleichzeitig überschaubarem Budget.

---

## Überprüfung zentraler Hypothesen

| **Hypothese** | **Beschreibung** | **Methode & Beobachtung** | **Ergebnis** |
|---------------|------------------|---------------------------|--------------|
| **1** | Tropisch warme Länder sind touristisch geprägt. | Preisvergleiche in Gastronomie, Freizeit und Kleidung. | Tropische Länder weisen überwiegend niedrigere Preise auf. |
| **2** | Lebenshaltungskosten sind dort geringer als in Europa/Nordamerika. | Clusteranalyse tropischer vs. nicht-tropischer Städte. | Tropische Städte dominieren die günstigsten Cluster. |
| **3** | Bessere Infrastruktur führt zu höheren Lebenshaltungskosten. | Scatterplot: Infrastrukturkosten vs. Mieten. | Positiver Trend, aber starke lokale Abweichungen. |
| **4** | Höheres Einkommen bedingt höhere Wohnkosten. | Korrelation: Einkommen vs. Mieten & Kaufpreise. | Sehr starke positive Korrelation. |
| **5** | ÖPNV-Kosten orientieren sich am lokalen Einkommen. | Regressionsanalyse: Einkommen vs. ÖPNV-Kosten. | Bestätigung: Höheres Einkommen → höhere ÖPNV-Kosten. |
| **6** | Freizeitkosten als Indikator für urbanen Lebensstandard. | Freizeit-Index und Regression mit Einkommen. | Klare positive Korrelation. |

---

## Zentrale Erkenntnisse

### Lebenshaltungskosten

- **Wohnen**: Besonders günstig in Medan, Bucaramanga und Pereira. Deutliche Einsparungen bei Miete im Zentrum wie außerhalb.
- **Lebensmittel**: Kolumbianische Städte bieten die niedrigsten Preise für frische Produkte und Grundnahrungsmittel.
- **Einkommen**: Lokale Gehälter sind irrelevant für Remote Worker, aber beeinflussen lokale Preisniveaus.

### Infrastruktur & Freizeit

- **Internet und Strom**: Colombo überzeugt mit hoher Qualität. In Teilen Afrikas und der Karibik treten Schwankungen auf.
- **Freizeitangebote**: Gute Verfügbarkeit und moderate Kosten für Sport, Kino und soziale Aktivitäten.
- **Öffentlicher Nahverkehr**: In fast allen analysierten tropischen Städten sehr kostengünstig nutzbar – besonders vorteilhaft für längere Aufenthalte ohne eigenes Fahrzeug.

---

## Top 5 Remote-Work-Städte

| **Stadt**        | **Land**       | **Besondere Stärken**                                                 |
|------------------|----------------|------------------------------------------------------------------------|
| Colombo          | Sri Lanka      | Sehr gutes Gesamtpaket: niedrige Kosten, stabile Infrastruktur, vielseitige Freizeitangebote |
| Bucaramanga      | Kolumbien      | Extrem günstige Lebenshaltungskosten, insbesondere bei Wohnen und Ernährung |
| Pereira          | Kolumbien      | Ausgewogene Preisstruktur, gutes Freizeitangebot                      |
| Medan            | Indonesien     | Beste Wohnkosten, stabile Infrastruktur, günstige Gastronomie         |
| Accra            | Ghana          | Gute Internetversorgung, preiswerte Freizeitaktivitäten, günstige Kleidung |

---

## Fazit

Die Daten belegen, dass tropisch-warme Städte nicht nur klimatisch, sondern auch ökonomisch äußerst attraktiv für Remote Work sind. Besonders Regionen in Süd- und Südostasien sowie Lateinamerika überzeugen durch ein hohes Preis-Leistungs-Verhältnis. 

Colombo hebt sich als Gesamtsieger hervor und bietet die besten Rahmenbedingungen für produktives Arbeiten bei gleichzeitig hoher Lebensqualität. Kolumbien stellt mit Bucaramanga und Pereira gleich zwei weitere Top-Optionen. Medan überzeugt bei Wohnkosten, Accra bietet eine starke afrikanische Alternative mit solider Infrastruktur. 

Die Analyse bestätigt: Für Remote Worker mit europäischem oder nordamerikanischem Einkommen stellen tropisch-warme Städte eine besonders effiziente Option dar, um Arbeit, Lebensqualität und Kosten in Einklang zu bringen.
